In [1]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [2]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [3]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [4]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, t.total_flow ,
t.avg_speed
from traffic t, pemslocs l
where t.timestamp between '2021-01-22 15:00:00' and '2021-01-22 15:03:00'
and l.sid = t.station and l.stype = 'ML'
"""
snap=pd.read_sql(query, engine, index_col="sid")
snap.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed
sid,,,,,,,,
1108285,-117.134145,32.713318,ML,94,W,0.819,232,64.8
1108287,-117.127908,32.714211,ML,94,W,1.205,153,58.6
1108289,-117.119831,32.716724,ML,94,W,1.721,252,62.9
1108291,-117.117861,32.717134,ML,94,W,1.844,311,65.1
1108293,-117.113171,32.717909,ML,94,W,2.134,277,62.8


In [5]:
pd.read_sql("select * from pemslocs limit 1", engine)
#print(len(snap))

,sid,fwy,direc,district,county,city,state_pm,abs_pm,latitude,longitude,length,stype,lanes,name
0,308511,50,E,3,17,None,31.627,60.162,38.761062,-120.569835,5.0,ML,2,Sly Park Rd


In [6]:
snap["total_flow"]=snap["total_flow"].astype(float)

In [7]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['94W', '125S', '78E', '163S', '8W', '8E', '125N', '94E', '805S',
       '78W', '15N', '5S', '15S', '5N', '805N', '163N', '54E', '54W',
       '56E', '56W', '52E', '52W', '67S', '67N', '905W', '905E', '76W',
       '76E', '11E', '11W'], dtype=object)

In [8]:
import pickle
with open("SD_fwys.pkl", "wb") as fid:
    pickle.dump(list(snap["fwdir"].unique()), fid)

In [154]:
fwdir='5S' #'94W'
sel=snap.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
sel.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed,fwdir
sid,,,,,,,,,
1114228,-117.032738,32.544296,ML,5,S,0.221,406.0,68.5,5S
1118326,-117.045806,32.551447,ML,5,S,1.140,169.0,66.7,5S
1118341,-117.061806,32.558188,ML,5,S,2.186,211.0,69.0,5S
1114721,-117.067228,32.561106,ML,5,S,2.559,304.0,66.4,5S
1118365,-117.076731,32.568952,ML,5,S,3.336,281.0,67.7,5S


In [156]:
selhr=pd.read_sql("select * from usdot u where u.fwydir = '%s' order by u.abs_pm" % fwdir, engine)
selhr.head()

from scipy.interpolate import interp1d
interpmod = interp1d(sel["abs_pm"].values, sel["avg_speed"].values, bounds_error=False)
selhr["avg_speed"] = interpmod(selhr["abs_pm"].values)
import numpy as np
selhr.dropna(axis="index", how="any", subset=["avg_speed"], inplace=True)
selhr["avg_speed"].describe()

count    1533.000000
mean       61.088765
std         6.300433
min        19.963853
25%        59.565145
50%        61.746141
75%        65.278844
max        70.299276
Name: avg_speed, dtype: float64

In [166]:
def speed2cat(speed):
    if speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    else:
        return 'very slow'
        
selhr["speed_cat"]=selhr["avg_speed"].apply(speed2cat)

In [160]:
selhr[0:10]

,usdid,fwydir,latitude,longitude,abs_pm,avg_speed,speed_cat
17,5318,5S,32.544431,-117.032924,0.226163,68.489887,fast
18,5319,5S,32.544469,-117.032980,0.230351,68.481686,fast
19,5320,5S,32.544525,-117.033062,0.236497,68.469647,fast
20,5321,5S,32.544693,-117.033312,0.255119,68.433174,fast
21,5322,5S,32.544749,-117.033395,0.261311,68.421046,fast
22,5323,5S,32.546390,-117.035804,0.441704,68.067718,fast
23,5324,5S,32.549000,-117.039847,0.738296,67.486797,fast
24,5325,5S,32.549081,-117.039993,0.748476,67.466859,fast
25,5326,5S,32.549325,-117.040435,0.779248,67.406588,fast
26,5327,5S,32.549586,-117.040948,0.814146,67.338235,fast


In [167]:
#split road(s) dataframe into groups with equal speed catergory
def road_group_split(indf, splitcat="speed_cat", keep_cols=["latitude", "longitude", "abs_pm", "avg_speed"]):
    
    odict={}
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selhrg=road_group_split(selhr)
selhrg.tail()

,latitude,longitude,abs_pm,avg_speed,speed_cat,linegroup
1573,33.378773,-117.566310,70.224601,63.856431,moderate,45
1574,33.378946,-117.566742,70.252244,63.849813,moderate,45
1575,33.379364,-117.567818,70.320714,63.833420,moderate,45
1576,33.380701,-117.571265,70.539999,63.780921,moderate,45
1577,33.381797,-117.574005,70.715284,63.738956,moderate,45


In [169]:
sel.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed,fwdir
sid,,,,,,,,,
1114228,-117.032738,32.544296,ML,5,S,0.221,406.0,68.5,5S
1118326,-117.045806,32.551447,ML,5,S,1.140,169.0,66.7,5S
1118341,-117.061806,32.558188,ML,5,S,2.186,211.0,69.0,5S
1114721,-117.067228,32.561106,ML,5,S,2.559,304.0,66.4,5S
1118365,-117.076731,32.568952,ML,5,S,3.336,281.0,67.7,5S


In [186]:
#del fig
fig = px.line_mapbox(selhrg, lat="latitude", lon="longitude", color="speed_cat",
                        mapbox_style="carto-darkmatter",
                        #color_continuous_scale="aggrnyl" ,
                        line_group = "linegroup",
                        hover_data=["abs_pm", "avg_speed"]) #"stype", 

fig2 = px.scatter_mapbox(sel, lat="latitude", lon="longitude",
                        hover_data=["stype","fwy","direc","abs_pm","total_flow","avg_speed"],
                         color="stype",
                        color_discrete_map={"ML": "white"},
                        opacity=0.35)
fig.add_trace(fig2.data[0])
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [174]:
px.scatter_mapbox?

In [56]:
import plotly.graph_objects as go

In [99]:
go.Scattermapbox?

In [127]:
fig2=px.scatter_mapbox(sel, lat="latitude", lon="longitude", mapbox_style="carto-darkmatter")


#tr=go.Scattermapbox(lat=selhr["latitude"], lon=selhr["longitude"],
#        mode='markers')
        #marker=go.scattermapbox.Marker(size=14))
#fig2.add_trace(tr)
#fig2.update_layout()
fig2

In [129]:
go.Scattermapbox?